# Day 1 - Function Calling

In this notebook, we explore the function calling feature offered by OpenAI's GPT models. While the name sounds pretty fancy, in essence, given the definition of a number of functions, the LLM can decide which function to call and return a JSON object that matches the required parameters. This functionality is achieved through finetuning. Note that while the model returns the parameters for the function call, the function itself is not called by the model. The user must manually call the function or utilize this API as part of a larger system.

You can read more about this function calling capability [here](https://platform.openai.com/docs/guides/gpt/function-calling).

Note: Since this is an OpenAI-specific functionality, we do not use the universal `llm_call` from previous notebooks. Instead, we directly use the OpenAI API.

In [1]:
# Load environment variables
from dotenv import load_dotenv

load_dotenv("../../.env")

True

In [5]:
import openai
import json

In [12]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="celsius"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "18",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [13]:
def run_conversation(message):
    # Step 1: send the conversation and available functions to GPT
    messages = [
        {"role": "system", "content": "You are a helpful chat assistant. Only use the functions you have been provided with."},
        {"role": "user", "content": message}
    ]
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]
    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        function_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        return second_response["choices"][0]["message"]["content"]
    else:
        return response_message["content"]

In [14]:
# Model detects that we need to make a function call here
print(run_conversation("What's the weather like in Toronto?"))

The weather in Toronto is currently 18°C, sunny, and windy.


In [15]:
# Model doesn't make any function calls here
print(run_conversation("When is the best time to visit Banff?"))

The best time to visit Banff, Canada, largely depends on the activities you're interested in. 

Summer (June to August) is a great time for hiking, wildlife viewing, and mountain biking. It's the busiest time of year, so you'll need to book accommodation and activities far in advance.

The winter season (December to March) offers excellent conditions for skiing, snowboarding, and other winter sports. Banff's three ski areas - Mount Norquay, Sunshine Village, and Lake Louise - usually have good snow conditions throughout the season.

Fall (September to November) can be a more peaceful time to visit, with fewer crowds. The weather can be unpredictable, so be prepared for various conditions. 

Spring (April to May) is a transition period with both winter snow and warmer days. Good for those looking for quieter times and considered off-peak.  

Keep in mind that mountain weather can be unpredictable, so it's always a good idea to check the current weather before you head out.


## Conclusion

While the above example appears relatively simple, it demonstrates how we can use function calling in more powerful ways. It converts natural language queries into a structured function call which can then be called and returned directly or used in further downstream systems. You can find a more detailed example [in the OpenAI Cookbook](https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models).